![Ejemplo 5](images/Ej5.png)

In [1]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

In [2]:
Hot_datos = {0: [0,0], 1: [600,0], 2: [120,120], 3: [280,280],4: [0,200], 5:[0,0]}
Hot_df = pd.DataFrame(Hot_datos,index=['H1','H2'])
Hot_df

,0,1,2,3,4,5
H1,0,600,120,280,0,0
H2,0,0,120,280,200,0


In [3]:
Cold_datos =  {0: [0], 1:[0], 2: [0], 3: [700], 4: [500], 5:[400]}
Cold_df = pd.DataFrame(Cold_datos,index=['C1'])
Cold_df

,0,1,2,3,4,5
C1,0,0,0,700,500,400


In [4]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
s=hotst.copy()
s.insert(0,'S')
w=coldst.copy()
w.insert(0,'W')

model.i=Set(initialize=hotst)
model.j=Set(initialize=coldst)
model.s=Set(initialize=s)
model.S=Set(initialize='S')
model.w=Set(initialize=w)
model.W=Set(initialize='W')
model.k=Set(initialize=Hot_df.columns)


In [5]:
model.pprint()

7 Set Declarations
    S : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :  {'S',}
    W : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :  {'W',}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'H1', 'H2'}
    j : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 : {'C1',}
    k : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    s : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'S', 'H1', 'H2'}
    w : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :

In [6]:
Qs = model.Qs = Var(model.k,within = NonNegativeReals) 
Qw = model.Qw = Var(model.k,within = NonNegativeReals) 
R = model.R = Var(model.s,model.k,within = NonNegativeReals)
Q = model.Q = Var(model.s,model.w,model.k,within = NonNegativeReals)

In [7]:
model.util = Objective(expr = model.Qs[1] + model.Qw[5])

In [8]:
nk = list(model.k)[1:] 
model.C1 = ConstraintList()
for k in nk:
    for i in model.i:
        model.C1.add(
        R[i,k-1]+Hot_df.loc[i,k] == R[i,k]+sum(Q[i,j,k] for j in model.j)+sum(Q[i,w,k] for w in model.W)
        )

model.C2 = ConstraintList()
for k in nk:
    for s in model.S:
        model.C2.add(
        R[s,k-1]+Qs[k] == R[s,k]+sum(Q[s,j,k] for j in model.j)
        )        

model.C3 = ConstraintList()
for k in nk:
    for j in model.j:
        model.C3.add(
        sum(Q[i,j,k] for i in model.i)+ sum(Q[i,j,k] for i in model.S) == Cold_df.loc[j,k]
        )

model.C4 = ConstraintList()
for k in nk:
        model.C4.add(
        sum(Q[i,'W',k] for i in model.i)  == Qw[k]
        )


model.C5 = ConstraintList()
for s in model.s:
    model.C5.add(
        R[s,0]==0
        )

model.C6 = ConstraintList()
for s in model.s:
    model.C6.add(
        R[s,5]==0
        )

nm = list(model.k)[0:1]+list(model.k)[2:]
model.C7 = ConstraintList()
for m in nm:
    model.C7.add(
        Qs[m]==0
        )

nn = list(model.k)[0:5]
model.C8 = ConstraintList()
for n in nn:
    model.C8.add(
        Qw[n]==0
        )

model.C9 = ConstraintList()
for k in model.k:
    model.C9.add(
        Q['S','W',k] ==0
    )

In [9]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

18 Set Declarations
    C1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C4_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C5_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    C6_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    C7_index : Size=1, Index=None, Ordered=Ins

In [10]:
Qss = value(model.Qs[1])
Qww = value(model.Qw[5])
print('Cold utility = {0:2.2f}, Hot utility = {1:2.2f}'.format(Qww, Qss))

Cold utility = -0.00, Hot utility = 0.00


Ahora resolvemos el problema de mínimo número de intercambios

In [11]:
U = {'C1': [0, 1000,600],  'W':[0,0,0]}
U_df = pd.DataFrame(U, index=['S', 'H1', 'H2'])
U_df

,C1,W
S,0,0
H1,1000,0
H2,600,0


In [12]:
model.y=Var(model.s,model.w, within=Binary)

model.C10 = ConstraintList()
for s in model.s:
    for w in model.w:
        model.C10.add(
        sum(Q[s,w,k] for k in model.k)-U_df.loc[s,w]*model.y[s,w] <=0
        )    

In [13]:
model.util.deactivate()
model.sumy = Objective(expr=sum(model.y[s,w] for s in model.s for w in model.w))

In [14]:
results = SolverFactory('glpk').solve(model)
#model.pprint()
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2.0
  Upper bound: 2.0
  Number of objectives: 1
  Number of constraints: 54
  Number of variables: 73
  Number of nonzeros: 151
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.020473957061767578
# ----------------------------------------------------------
#   Solution Information
# ------------------------------------

![Ejemplo 5](images/Ej5_2.png)

In [15]:
modelNLP = ConcreteModel()

In [16]:
""" f1I = modelNLP.f1I = Var(bounds=(0,10), initialize=2.942)
f2I = modelNLP.f2I = Var(bounds=(0,10),initialize=7.058)
f1O = modelNLP.f1O = Var(bounds=(0,10),initialize=2.942)
f2O = modelNLP.f2O = Var(bounds=(0,10),initialize=7.058)
f1E = modelNLP.f1E = Var(bounds=(0,10),initialize=2.942)
f2E = modelNLP.f2E = Var(bounds=(0,10),initialize=7.058)
f12B = modelNLP.f12B = Var(bounds=(0,10),initialize=0)
f21B = modelNLP.f21B = Var(bounds=(0,10),initialize=0)

T1I = modelNLP.T1I = Var(bounds=(150,500),initialize=150)
T2I = modelNLP.T2I = Var(bounds=(150,500),initialize=150)
T1O = modelNLP.T1O = Var(bounds=(150,500),initialize=150)
T2O = modelNLP.T2O = Var(bounds=(150,500),initialize=150)

DT11 = modelNLP.DT11 = Var(bounds=(10,500),initialize=10)
DT12 = modelNLP.DT12 = Var(bounds=(10,500),initialize=10)
DT21 = modelNLP.DT21 = Var(bounds=(10,500),initialize=10)
DT22 = modelNLP.DT22 = Var(bounds=(10,500),initialize=10) """

f1I = modelNLP.f1I = Var(bounds=(0,10), initialize=10)
f2I = modelNLP.f2I = Var(bounds=(0,10),initialize=10)
f1O = modelNLP.f1O = Var(bounds=(0,10),initialize=10)
f2O = modelNLP.f2O = Var(bounds=(0,10),initialize=10)
f1E = modelNLP.f1E = Var(bounds=(0,10),initialize=10)
f2E = modelNLP.f2E = Var(bounds=(0,10),initialize=10)
f12B = modelNLP.f12B = Var(bounds=(0,10),initialize=10)
f21B = modelNLP.f21B = Var(bounds=(0,10),initialize=10)

T1I = modelNLP.T1I = Var(bounds=(150,500),initialize=500)
T2I = modelNLP.T2I = Var(bounds=(150,500),initialize=500)
T1O = modelNLP.T1O = Var(bounds=(150,500),initialize=500)
T2O = modelNLP.T2O = Var(bounds=(150,500),initialize=500)

DT11 = modelNLP.DT11 = Var(bounds=(10,500),initialize=500)
DT12 = modelNLP.DT12 = Var(bounds=(10,500),initialize=500)
DT21 = modelNLP.DT21 = Var(bounds=(10,500),initialize=500)
DT22 = modelNLP.DT22 = Var(bounds=(10,500),initialize=500)

In [17]:
modelNLP.SPLIT1 = Constraint(expr=f1I+f2I==10)
modelNLP.SPLIT2 = Constraint(expr=f1O+f12B-f1E==0)
modelNLP.SPLIT3 = Constraint(expr=f2O+f21B-f2E==0)
modelNLP.MIX1 = Constraint(expr=f1I+f21B-f1E==0)
modelNLP.MIX2 = Constraint(expr=f2I+f12B-f2E==0)
modelNLP.MIX3 = Constraint(expr=f1O+f2O==10)


modelNLP.EN_MIX1 = Constraint(expr=150*f1I+T2O*f21B-T1I*f1E==0)
modelNLP.EN_MIX2 = Constraint(expr=150*f2I+T1O*f12B-T2I*f2E==0)
modelNLP.EN_MIX3 = Constraint(expr=T2O*f2O+T1O*f1O-310*10==0)

modelNLP.HEATEX1 = Constraint(expr=f1E*(T1O-T1I)==1000)
modelNLP.HEATEX2 = Constraint(expr=f2E*(T2O-T2I)==600)

modelNLP.D11 = Constraint(expr=500-T1O>=10)
modelNLP.D12 = Constraint(expr=250-T1I>=10)
modelNLP.D21 = Constraint(expr=350-T2O>=10)
modelNLP.D22 = Constraint(expr=200-T2I>=10)

modelNLP.DELT11 = Constraint(expr=500-T1O==DT11)
modelNLP.DELT12 = Constraint(expr=250-T1I==DT12)
modelNLP.DELT21 = Constraint(expr=350-T2O==DT21)
modelNLP.DELT22 = Constraint(expr=200-T2I==DT22)

In [18]:
modelNLP.Coste = Objective(expr=1300*(1000/(0.05*(2/3*(DT11*DT12)**0.5+1/6*(DT11+DT12))))**0.6+1300*(600/(0.05*(2/3*(DT21*DT22)**0.5+1/6*(DT21+DT22))))**0.6)

In [19]:
solver = 'ipopt'
resultsNLP = SolverFactory(solver).solve(modelNLP)
#modelNLP.pprint()
resultsNLP.write()
Cost = value(modelNLP.Coste)
Area1 = 1000/(0.05*(2/3*(value(DT11)*value(DT12))**0.5+1/6*(value(DT11)+value(DT12))))
Area2 = 600/(0.05*(2/3*(value(DT21)*value(DT22))**0.5+1/6*(value(DT21)+value(DT22))))

print('El coste usando {0} es de {1:.2f}'.format(solver,Cost))
print('El área del primer cambiador es {0:.2f}'.format(Area1))
print('El área del segundo cambiador es {0:.2f}'.format(Area2))
print('La cantidad que se manda al cambiador 1 es {0:.2f}'.format(value(modelNLP.f1I)))
print('La cantidad que entra al cambiador 1 es {0:.2f}'.format(value(modelNLP.f1E)))
print('La cantidad que se manda al cambiador 2 es {0:.2f}'.format(value(modelNLP.f2I)))
print('La cantidad que entra al cambiador 2 es {0:.2f}'.format(value(modelNLP.f2E)))

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 19
  Number of variables: 16
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.17274713516235352
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
El cos

In [22]:
solver = 'multistart'
resultsNLP = SolverFactory(solver).solve(modelNLP)
#modelNLP.pprint()
resultsNLP.write()
Cost = value(modelNLP.Coste)
Area1 = 1000/(0.05*(2/3*(value(DT11)*value(DT12))**0.5+1/6*(value(DT11)+value(DT12))))
Area2 = 600/(0.05*(2/3*(value(DT21)*value(DT22))**0.5+1/6*(value(DT21)+value(DT22))))

print('El coste usando {0} es de {1:.2f}'.format(solver,Cost))
print('El área del primer cambiador es {0:.2f}'.format(Area1))
print('El área del segundo cambiador es {0:.2f}'.format(Area2))
print('La cantidad que se manda al cambiador 1 es {0:.2f}'.format(value(modelNLP.f1I)))
print('La cantidad que entra al cambiador 1 es {0:.2f}'.format(value(modelNLP.f1E)))
print('La cantidad que se manda al cambiador 2 es {0:.2f}'.format(value(modelNLP.f2I)))
print('La cantidad que entra al cambiador 2 es {0:.2f}'.format(value(modelNLP.f2E)))

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 19
  Number of variables: 16
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02399444580078125
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
El cos

In [21]:
solver = 'gams'
options = {}
#resultsNLP = SolverFactory(solver).solve(modelNLP, solver='baron', keepfiles=True, tee=True, add_options=['option optcr=1e-9;', 'option optca=0.001;', 'option NLP=baron;'])
resultsNLP = SolverFactory('gams').solve(modelNLP, tee=True, solver='baron', add_options = ['GAMS_MODEL.optfile = 1;','$onecho > baron.opt', ' NLPSol= 6', 'ExtNLPsolver=conopt4', 'optcr=0.0001','optcr=0.0001','NumSol=10', '$offecho'])
#modelNLP.pprint()
resultsNLP.write()
Cost = value(modelNLP.Coste)
Area1 = 1000/(0.05*(2/3*(value(DT11)*value(DT12))**0.5+1/6*(value(DT11)+value(DT12))))
Area2 = 600/(0.05*(2/3*(value(DT21)*value(DT22))**0.5+1/6*(value(DT21)+value(DT22))))

print('El coste usando {0} es de {1:.2f}'.format(solver,Cost))
print('El área del primer cambiador es {0:.2f}'.format(Area1))
print('El área del segundo cambiador es {0:.2f}'.format(Area2))
print('La cantidad que se manda al cambiador 1 es {0:.2f}'.format(value(modelNLP.f1I)))
print('La cantidad que entra al cambiador 1 es {0:.2f}'.format(value(modelNLP.f1E)))
print('La cantidad que se manda al cambiador 2 es {0:.2f}'.format(value(modelNLP.f2I)))
print('La cantidad que entra al cambiador 2 es {0:.2f}'.format(value(modelNLP.f2E)))

--- Job model.gms Start 10/02/23 17:34:43 25.0.0 r65090 WEX-WEI x86 64bit/MS Windows
GAMS 25.0.0   Copyright (C) 1987-2018 GAMS Development. All rights reserved
*** ************* BETA release
*** GAMS Base Module 25.0.0 r65090 BETA Released 12Jan18 WEI x86 64bit/MS Windows 
*** ************* BETA release
Licensee: Small MUD - 5 User License                     S170124:0844AO-WIN
          Universidad Politecnica de Madrid, INGENIERIA QUIMICA INDUDC13043
          License for teaching and research at degree granting institutions
          License Admin: Ismael Diaz, ismael.diaz@upm.es                   
--- Starting compilation
--- model.gms(220) 2 Mb
--- Starting execution: elapsed 0:00:00.065
--- model.gms(119) 3 Mb
--- Generating NLP model GAMS_MODEL
--- model.gms(130) 5 Mb
--- Reset Solvelink = 2
---   20 rows  17 columns  53 non-zeroes
---   67 nl-code  22 nl-non-zeroes
--- model.gms(130) 3 Mb
--- Executing BARON (Solvelink=2): elapsed 0:00:00.166

GAMS/BARON       25.0.0 r65090 BE